# 電力データを変換する

In [1]:
import random
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp, to_date, explode
from datetime import datetime, timedelta

In [2]:
data_file = 'data/power_grid_sample.csv'
randomized_file = 'data/power_grid_dummydata'

In [3]:
df1 = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(data_file)

In [4]:
df1.schema

StructType(List(StructField(DATETIME,StringType,true),StructField(Measured,IntegerType,true),StructField(Predicted,IntegerType,true),StructField(UseRate,IntegerType,true),StructField(EstimatedSupplyCapacity,IntegerType,true)))

In [5]:
df2 = df1.select(to_timestamp(df1['DATETIME'], 'yyyy/M/d H:mm').alias('timestamp'), '*')

In [6]:
randomize_width = 30

In [7]:
@udf(IntegerType())
def randamize(col):
    diff = random.randint(-randomize_width, randomize_width)
    return col + diff

In [8]:
df3 = df2.select(randamize(df2['Measured']).alias('RMeasured'),
                randamize(df2['Predicted']).alias('RPredicted'),
                randamize(df2['EstimatedSupplyCapacity']).alias('REstimatedSupplyCapacity'), '*')

In [9]:
df4 = df3.select((df3['RMeasured']/df3['REstimatedSupplyCapacity']).alias('RUseRate'), '*')

In [10]:
df5 = df4.select('timestamp', 'RMeasured', 'RPredicted', 'RUseRate', 'REstimatedSupplyCapacity')

In [20]:
@udf(ArrayType(TimestampType()))
def transform_timestamp(timestamp):
    records = [timestamp]
    num_records = 59
    for i in range(0, num_records):
        delta = timedelta(minutes=i+1)
        records.append(timestamp + delta)
        
    return records

In [21]:
df6 = df5.select(explode(transform_timestamp(df5['timestamp'])).alias('timestamp_min'), '*')

In [22]:
df6.show(70)

+-------------------+-------------------+---------+----------+------------------+------------------------+
|      timestamp_min|          timestamp|RMeasured|RPredicted|          RUseRate|REstimatedSupplyCapacity|
+-------------------+-------------------+---------+----------+------------------+------------------------+
|2021-01-05 00:00:00|2021-01-05 00:00:00|     1486|      1500|0.8089275993467611|                    1837|
|2021-01-05 00:01:00|2021-01-05 00:00:00|     1486|      1500|0.8089275993467611|                    1837|
|2021-01-05 00:02:00|2021-01-05 00:00:00|     1486|      1500|0.8089275993467611|                    1837|
|2021-01-05 00:03:00|2021-01-05 00:00:00|     1486|      1500|0.8089275993467611|                    1837|
|2021-01-05 00:04:00|2021-01-05 00:00:00|     1486|      1500|0.8089275993467611|                    1837|
|2021-01-05 00:05:00|2021-01-05 00:00:00|     1486|      1500|0.8089275993467611|                    1837|
|2021-01-05 00:06:00|2021-01-05 00:00

In [23]:
randomize_width = 10

In [24]:
df7 = df6.select(randamize(df6['RMeasured']).alias('RRMeasured'), '*')

In [25]:
df7.show()

+----------+-------------------+-------------------+---------+----------+------------------+------------------------+
|RRMeasured|      timestamp_min|          timestamp|RMeasured|RPredicted|          RUseRate|REstimatedSupplyCapacity|
+----------+-------------------+-------------------+---------+----------+------------------+------------------------+
|      1505|2021-01-05 00:00:00|2021-01-05 00:00:00|     1476|      1502|0.7918454935622318|                    1864|
|      1491|2021-01-05 00:01:00|2021-01-05 00:00:00|     1476|      1502|0.7918454935622318|                    1864|
|      1492|2021-01-05 00:02:00|2021-01-05 00:00:00|     1476|      1502|0.7918454935622318|                    1864|
|      1493|2021-01-05 00:03:00|2021-01-05 00:00:00|     1476|      1502|0.7918454935622318|                    1864|
|      1456|2021-01-05 00:04:00|2021-01-05 00:00:00|     1476|      1502|0.7918454935622318|                    1864|
|      1506|2021-01-05 00:05:00|2021-01-05 00:00:00|    

In [26]:
df8 = df7.select((df7['RRMeasured']/df7['REstimatedSupplyCapacity']).alias('RRUseRate'), '*')

In [27]:
df8.show()

+------------------+----------+-------------------+-------------------+---------+----------+-----------------+------------------------+
|         RRUseRate|RRMeasured|      timestamp_min|          timestamp|RMeasured|RPredicted|         RUseRate|REstimatedSupplyCapacity|
+------------------+----------+-------------------+-------------------+---------+----------+-----------------+------------------------+
|               0.8|      1480|2021-01-05 00:00:00|2021-01-05 00:00:00|     1478|      1507|0.798918918918919|                    1850|
|0.7854054054054054|      1453|2021-01-05 00:01:00|2021-01-05 00:00:00|     1478|      1507|0.798918918918919|                    1850|
|0.7908108108108108|      1463|2021-01-05 00:02:00|2021-01-05 00:00:00|     1478|      1507|0.798918918918919|                    1850|
|0.8145945945945946|      1507|2021-01-05 00:03:00|2021-01-05 00:00:00|     1478|      1507|0.798918918918919|                    1850|
|0.8118918918918919|      1502|2021-01-05 00:04:

In [28]:
df9 = df8.select(df8['timestamp_min'].alias('timestamp'), df8['RRMeasured'].alias('Measured'), df8['RPredicted'].alias('Predicted'), df8['RRUseRate'].alias('UseRate'), df8['REstimatedSupplyCapacity'].alias('EstimatedSupplyCapacity'))

In [29]:
df9.show()

+-------------------+--------+---------+------------------+-----------------------+
|          timestamp|Measured|Predicted|           UseRate|EstimatedSupplyCapacity|
+-------------------+--------+---------+------------------+-----------------------+
|2021-01-05 00:00:00|    1475|     1533|0.7858284496537027|                   1877|
|2021-01-05 00:01:00|    1510|     1533|0.8044752264251465|                   1877|
|2021-01-05 00:02:00|    1521|     1533| 0.810335641981886|                   1877|
|2021-01-05 00:03:00|    1507|     1533|0.8028769312733085|                   1877|
|2021-01-05 00:04:00|    1509|     1533|0.8039424613745338|                   1877|
|2021-01-05 00:05:00|    1523|     1533|0.8114011720831114|                   1877|
|2021-01-05 00:06:00|    1522|     1533|0.8108684070324986|                   1877|
|2021-01-05 00:07:00|    1523|     1533|0.8114011720831114|                   1877|
|2021-01-05 00:08:00|    1520|     1533|0.8098028769312733|                 

In [30]:
df9.write.format('csv').option('header', 'true').option('sep', r'/t').save(randomized_file+'.tsv')
df9.write.format('delta').save(randomized_file+'.delta')